In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup, default_opt_set
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21, modified_Smolarkiewicz_Rasch_r0
from MPyDATA_examples.Olesik_et_al_2020.convergence_plotter  import polar_plot

In [3]:
GCs = np.linspace(.05,.95, 5)
nrs = np.array([2**n for n in range(10, 14)], dtype=int)

In [4]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [5]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    psi = simulation.psi
    g_factor = simulation.g_factor
    dp_dr = simulation.dp_dr

    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    dif = analytical-numerical
    measure_h_inf = ((maximum_numeric - maximum_analytic) / maximum_analytic).magnitude
    measure_h_2 =  (1/t * np.sqrt( 1/len(dif) * dif.dot(dif))).magnitude
    
    def moment_integral(x, k):
        if psi_coord.__class__.__name__ == 'x_id':
            return 1 / (k + 1) * x**(k+1)
        elif psi_coord.__class__.__name__ == 'x_p2':
            return 2 / (k + 2) * x**((k+2)/2)
        elif psi_coord.__class__.__name__ == 'x_p3':
            return 3 / (k + 3) * x**((k+3)/3)
        elif psi_coord.__class__.__name__ == 'x_log_of_pn':
            return 1 / k * np.exp(k * x)  #TODO: not properly implemented because it's not natural log usually
    
    def rel_disp(r, psi, g_factor):
        #TODO: move to analysis/simulation and write over simulation plots
        mom0 = 0
        mom1 = 0
        mom2 = 0
        for i in range(len(psi)):
            psi_i = psi[i] 
            dp_i = moment_integral(psi_coord.x(r[i+1]),0) - moment_integral(psi_coord.x(r[i]),0)
            A_i = moment_integral(psi_coord.x(r[i+1]),1) - moment_integral(psi_coord.x(r[i]),1)
            B_i = moment_integral(psi_coord.x(r[i+1]),2) - moment_integral(psi_coord.x(r[i]),2)
            bin_mom0= psi_i * dp_i
            bin_mom1 = psi_i * A_i
            bin_mom2 = psi_i * B_i
            mom0+=bin_mom0
            mom1+=bin_mom1
            mom2+=bin_mom2
        mu = mom1 / mom0
        normalised_mom2 = mom2 / mom0
        print(normalised_mom2)
        print(mu)
        std = np.sqrt(normalised_mom2 - mu**2)
        d = std/mu
        return d
    numeric_rel_d = rel_disp(rh, numerical/dp_dr, g_factor)
    analytic_rel_d = rel_disp(rh, analytical/dp_dr, g_factor)
    rel_dispersion = numeric_rel_d / analytic_rel_d 
    print("analytic rd", analytic_rel_d)
    print("numeric rd", numeric_rel_d)

    
    
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    error_g = np.log2(Smolarkiewicz_Grabowski_1990_eq21(g_factor * psi.magnitude, g_factor * analytical.magnitude / dp_dr, t.magnitude))
    error_r0 = np.log2(modified_Smolarkiewicz_Rasch_r0(psi.magnitude, analytical.magnitude /dp_dr, t.magnitude, g_factor))
    print("error old", error)
    print("new error (including G)", error_g)
    print("new error S&R(including G), r0", error_r0)
    print("\n")


    return setup.nr, GC, error, error_g, error_r0, measure_height #, measure_h_inf, measure_h_2 #, rel_dispersion

In [6]:
opts = default_opt_set
def replace_names(opt):
    str_repl = [["'n_iters': 1","upwind"],
                ["'n_iters': 2","MPDATA 2 iterations"],
                ["'n_iters': 3","MPDATA 3 iterations"],
                ["'",""],
                [": True",""],
                ["_", " "],
                ["{",""],["}",""],[","," "]]                            
    for repl in str_repl:
        opt = str(opt).replace(repl[0], repl[1])
    return opt

In [ ]:
for opt in opts:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2] , 'log$_2$(err_g)':results[3] , 'log$_2$(err_r0)':results[4]} # , 'height difference':results[3], 'h_inf':results[4], 'h_2': results[5]} #,'rel_disp':results[6]}
    opt = replace_names(opt)
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')   #TODO: dont show plot name when savingfig (for masters)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


64.88914049454355 micrometer ** 2
7.626311506839877 micrometer
64.88912948426676 micrometer ** 2
7.627092226565027 micrometer
analytic rd 0.33979386972283987 dimensionless
numeric rd 0.34013006087753955 dimensionless
error old -8.817775273584529
new error (including G) -11.727990546322511
new error S&R(including G), r0 -14.32232514349311


64.95004051540064 micrometer ** 2
7.6309209920523715 micrometer
64.95001979167338 micrometer ** 2
7.631529563006289 micrometer
analytic rd 0.3394239087225427 dimensionless
numeric rd 0.33968637009278685 dimensionless
error old -9.204697509995727
new error (including G) -12.110536918104115
new error S&R(including G), r0 -14.704887506744546


